In [1]:
import data_api3.reader as reader
import pandas as pd

In [131]:
def wrap(data: dict) -> pd.DataFrame:
    
    dataframe = None
    
    for key in data:
        df = pd.DataFrame(data[key])
        df = df.drop(columns=["pulse_id"])  # were not interested in this
        df = df.set_index('timestamp')  # set timestamp as index

        if dataframe is None:
            dataframe = df
        else:
            dataframe = pd.merge(dataframe, df, how='outer', on='timestamp')
            
    dataframe.fillna(method='pad', inplace=True)  # fill NaN with last known value (assuming recording system worked without error)

    return dataframe
            

In [2]:
query = {
  "channels": ["MHC1:IST:2", "MHC2:IST:2"],
  "range": {
    "type": "date",
    "startDate": "2019-11-06T10:00:00.000000000Z",
    "endDate": "2019-11-06T10:10:00.000000000Z"
  }
}


In [3]:
data = reader.request(query, url="http://hipa-daq-01:8090/api/v1/query")

25, 25


In [5]:
data.keys()

dict_keys(['MHC1:IST:2', 'MHC2:IST:2'])

In [96]:
df1 = pd.DataFrame(data["MHC1:IST:2"])

In [97]:
df1 = df1.drop(columns=["pulse_id"])
df1.head()

,timestamp,MHC1:IST:2
0,1573034400006541420,1819.7
1,1573034400206541420,1819.1
2,1573034400406541420,1819.5
3,1573034400606541420,1820.0
4,1573034400806541420,1819.9


In [98]:
df1 = df1.set_index('timestamp')
len(df1)

2804

In [99]:
df2 = pd.DataFrame(data["MHC2:IST:2"])

In [100]:
df2 = df2.drop(columns=["pulse_id"])
df2.head()

,timestamp,MHC2:IST:2
0,1573034400006541420,1824.8
1,1573034400206541420,1824.7
2,1573034400406541420,1824.4
3,1573034400606541420,1824.7
4,1573034400806541420,1824.5


In [101]:
df2 = df2.set_index('timestamp')
len(df2)

2793

In [102]:
r = pd.merge(df1, df2, how='outer', on='timestamp')

In [103]:
len(r)

2981

In [107]:
r.sort_index(inplace=True)
r

,MHC1:IST:2,MHC2:IST:2
timestamp,,
1573034400006541420,1819.7,1824.8
1573034400206541420,1819.1,1824.7
1573034400406541420,1819.5,1824.4
1573034400606541420,1820.0,1824.7
1573034400806541420,1819.9,1824.5
...,...,...
1573034999008022410,1819.8,1824.1
1573034999208022410,1821.8,1826.3
1573034999408022410,1820.0,1824.9


In [104]:
df2.sort_index(inplace=True)
df2.tail()

,MHC2:IST:2
timestamp,
1573034999008022410,1824.1
1573034999208022410,1826.3
1573034999408022410,1824.9
1573034999608022410,1823.8
1573034999808022410,1824.8


In [105]:
df1.sort_index(inplace=True)
df1.tail()

,MHC1:IST:2
timestamp,
1573034999008022410,1819.8
1573034999208022410,1821.8
1573034999408022410,1820.0
1573034999608022410,1819.5
1573034999808022410,1820.4


In [84]:
df2.loc[[1573034990408022410]]

,MHC2:IST:2
timestamp,
1573034990408022410,1825.6


In [93]:
len(set(df1['timestamp']) & set(df2['timestamp']))

2616

In [94]:
len(set(df1['timestamp']) | set(df2['timestamp']))

2981

In [115]:
pd.isna(r["MHC1:IST:2"]).value_counts()

False    2804
True      177
Name: MHC1:IST:2, dtype: int64

In [116]:
pd.isna(r["MHC2:IST:2"]).value_counts()

False    2793
True      188
Name: MHC2:IST:2, dtype: int64

In [117]:
r[pd.isna(r["MHC1:IST:2"]) | pd.isna(r["MHC2:IST:2"])]

,MHC1:IST:2,MHC2:IST:2
timestamp,,
1573034401806541420,1820.0,NaN
1573034402206541420,NaN,1825.5
1573034404006541420,NaN,1825.5
1573034404206541420,1820.1,NaN
1573034406406541420,1820.1,NaN
...,...,...
1573034982808022410,NaN,1824.9
1573034983408022410,NaN,1824.1
1573034990408022410,NaN,1825.6


In [122]:
r.fillna(method='pad', inplace=True)  # fill NaN with last known value (assuming recording system worked without error)

In [124]:
r.loc[[1573034982808022410, 1573034983408022410]]

,MHC1:IST:2,MHC2:IST:2
timestamp,,
1573034982808022410,1820.4,1824.9
1573034983408022410,1819.4,1824.1


In [132]:
wrap(data)

,MHC1:IST:2,MHC2:IST:2
timestamp,,
1573034400006541420,1819.7,1824.8
1573034400206541420,1819.1,1824.7
1573034400406541420,1819.5,1824.4
1573034400606541420,1820.0,1824.7
1573034400806541420,1819.9,1824.5
...,...,...
1573034975406839130,1820.4,1825.4
1573034979008022410,1820.4,1825.7
1573034982808022410,1820.4,1824.9


In [4]:
reader.as_dataframe(data)

,MHC1:IST:2,MHC2:IST:2
timestamp,,
1573034400006541420,1819.7,1824.8
1573034400206541420,1819.1,1824.7
1573034400406541420,1819.5,1824.4
1573034400606541420,1820.0,1824.7
1573034400806541420,1819.9,1824.5
...,...,...
1573034975406839130,1820.4,1825.4
1573034979008022410,1820.4,1825.7
1573034982808022410,1820.4,1824.9
